In [2]:
import math
from pathlib import Path

import arcpy
from ba_tools import data
import pandas as pd
import numpy as np

from h3 import h3
from itertools import chain
import json
import os
from pathlib import Path
import sys
from copy import deepcopy

from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

project_parent = Path('./').absolute().parent

data_dir = project_parent/'data'

data_raw = data_dir/'raw'
data_int = data_dir/'interim'
data_out = data_dir/'processed'

gdb_raw = data_raw/'raw.gdb'
gdb_int = data_int/'interim.gdb'
gdb_out = data_out/'processed.gdb'

# import the project package from the project package path
sys.path.append(str(project_parent/'src'))
import h3_arcgis

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

In [4]:
# pth_bp = Path(r'D:\arcgis\ba_data\Data\Demographic Data\block_data.gdb\USA_ESRI_2019_blocks')
pth_bp = gdb_int/'sea_block_points'
pth_bp

df_bp = GeoAccessor.from_featureclass(str(pth_bp))
df_bp.head()

df_bp = df_bp[['FIPS', 'SHAPE', 'HH_C', 'POP_C']].copy()
df_bp.columns = ['FIPS', 'SHAPE', 'HouseholdCount', 'Population']
df_bp.head()

df_bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45546 entries, 0 to 45545
Data columns (total 4 columns):
FIPS              45546 non-null object
SHAPE             45546 non-null geometry
HouseholdCount    45546 non-null float64
Population        45546 non-null float64
dtypes: float64(2), geometry(1), object(1)
memory usage: 1.4+ MB


In [12]:
spatially_enabled_dataframe = df_bp
h3_maximum = 9
h3_minimum = 5
minimum_threshold_value = 1000
weighting_column = 'Population'

# add h3 ids
df = h3_arcgis.add_h3_ids_to_points(spatially_enabled_dataframe, h3_maximum, h3_minimum)

# assign h3 grid based on count, and drop those blow the threshold
cnt_df = h3_arcgis.get_h3_ids_by_count(df, minimum_threshold_value, count_column=weighting_column)

cnt_df

,FIPS,SHAPE,HouseholdCount,Population,h3_09,h3_08,h3_07,h3_06,h3_05,h3_id,h3_lvl,h3_orig
0,530530729061083,"{'x': -122.70289379994466, 'y': 47.07056900018...",0.0,0.0,8928d59b63bffff,8828d59b63fffff,8728d59b6ffffff,8628d59b7ffffff,8528d59bfffffff,NaN,NaN,NaN
1,530530730061116,"{'x': -122.38923279983811, 'y': 46.84712710009...",2.0,6.0,8928f37843bffff,8828f37843fffff,8728f3784ffffff,8628f3787ffffff,8528f37bfffffff,8528f37bfffffff,5.0,5.0
2,530530732003038,"{'x': -122.27535699972861, 'y': 46.83756129998...",8.0,15.0,8928f34cd77ffff,8828f34cd7fffff,8728f34cdffffff,8628f34cfffffff,8528f34ffffffff,8628f34cfffffff,6.0,6.0
3,530530732003040,"{'x': -122.31008460020831, 'y': 46.84448979965...",2.0,4.0,8928f37b223ffff,8828f37b23fffff,8728f37b2ffffff,8628f37b7ffffff,8528f37bfffffff,8528f37bfffffff,5.0,5.0
4,530530732003041,"{'x': -122.25264919960182, 'y': 46.84812859970...",1.0,1.0,8928f34c83bffff,8828f34c83fffff,8728f34c8ffffff,8628f34cfffffff,8528f34ffffffff,8628f34cfffffff,6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45541,530330328003455,"{'x': -121.16257230027668, 'y': 47.71029490015...",2.0,3.0,8928d0d2e93ffff,8828d0d2e9fffff,8728d0d2effffff,8628d0d2fffffff,8528d0d3fffffff,NaN,NaN,NaN
45542,530330328003475,"{'x': -121.1590584001201, 'y': 47.710500499977...",0.0,0.0,8928d0d2167ffff,8828d0d217fffff,8728d0d21ffffff,8628d0d27ffffff,8528d0d3fffffff,NaN,NaN,NaN
45543,530330328003476,"{'x': -121.15500529956404, 'y': 47.71068230009...",0.0,0.0,8928d0d2167ffff,8828d0d217fffff,8728d0d21ffffff,8628d0d27ffffff,8528d0d3fffffff,NaN,NaN,NaN
45544,530330328003477,"{'x': -121.16192290033126, 'y': 47.71030499956...",1.0,2.0,8928d0d2e93ffff,8828d0d2e9fffff,8728d0d2effffff,8628d0d2fffffff,8528d0d3fffffff,NaN,NaN,NaN


In [16]:
# roll up smaller grid id assignments to larger parent hexbins
cln_cnt_df = h3_arcgis.remove_overlapping_h3_ids(cnt_df)

out_df = h3_arcgis.get_h3_hexbins_with_counts(cln_cnt_df, count_column_name=weighting_column)
out_df.head()

,h3_id,SHAPE,Population,h3_lvl
0,8528d557fffffff,"{'rings': [[[-122.36929827899996, 47.500677578...",1862,5.0
1,8528d0b7fffffff,"{'rings': [[[-122.20382834499998, 47.866673952...",1555,5.0
2,8628d5427ffffff,"{'rings': [[[-122.26493069499998, 47.570222506...",946,6.0
3,8528d437fffffff,"{'rings': [[[-121.96960155399995, 47.252683501...",809,5.0
4,8528d4affffffff,"{'rings': [[[-121.96013460199998, 47.111783662...",710,5.0


In [17]:
out_df.spatial.to_featureclass(str(gdb_int/f'sea_hex'))

'D:\\projects\\h3-py\\data\\interim\\interim.gdb\\sea_hex'